In [1]:
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from nltk.corpus import stopwords

import os
from sc_ft import *
from lm_ft import *

Using TensorFlow backend.


In [2]:
logger = logging.getLogger(__name__)
epoch_macro= 1

MODEL_CLASSES = {
    "openai-gpt": (OpenAIGPTConfig, OpenAIGPTLMHeadModel, OpenAIGPTTokenizer),
    "bert": (BertConfig, BertForMaskedLM, BertTokenizer),
    "roberta": (RobertaConfig, RobertaForMaskedLM, RobertaTokenizer),
}

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1060 6GB


In [3]:
class Args_LM:
    adam_epsilon=1e-8
    block_size=-1 
    cache_dir=""
    config_name=""
    do_train=True        
    do_eval=True 
    do_lower_case=True
    eval_all_checkpoints=False
    evaluate_during_training=False
    eval_data_file=None
    fp16= False
    fp16_opt_level="O1"
    gradient_accumulation_steps=1
    learning_rate=5e-5
    logging_steps=100
    local_rank=-1
    model_type= default="bert"
    model_name_or_path= "bert-base-cased" 
    mlm=True
    mlm_probability=0.15
    max_grad_norm=1.0
    max_steps=-1
    num_train_epochs=1.0
    n_gpu=1
    no_cuda=False
    overwrite_output_dir=True
    overwrite_cache=False
    output_dir=None
    per_gpu_train_batch_size=4
    per_gpu_eval_batch_size=4
    save_steps=100
    save_total_limit=None    
    seed=42   
    server_ip=""
    server_port=""
    train_data_file=None #str
    tokenizer_name="" 
    weight_decay=0.0   
    warmup_steps=0

In [4]:
out= '../../../../model_save/Dos-Fases-all_Harassment/'

my_args = Args_LM()
my_args.output_dir = out
my_args.model_type = 'bert'
my_args.train_data_file = '../../CSV_Harassment/Train_data_harassment.txt'
my_args.block_size = 64
my_args.eval_data_file = '../../CSV_Harassment/Test_data_harassment.txt'
my_args.per_gpu_train_batch_size = 16
my_args.per_gpu_eval_batch_size = 16

In [5]:
def fase_LM(from_pre, my_args, logger):
    if from_pre:
        model_dir_ft = 'bert-base-uncased'
        my_args.model_name_or_path = model_dir_ft
    else:
        model_dir_ft = out
        my_args.model_name_or_path = model_dir_ft
    
    if my_args.model_type in ["bert", "roberta"] and not my_args.mlm:
        raise ValueError(
            "BERT and RoBERTa do not have LM heads but masked LM heads. They must be run using the --mlm "
            "flag (masked language modeling)."
        )
    if my_args.eval_data_file is None and my_args.do_eval:
        raise ValueError(
            "Cannot do evaluation without an evaluation data file. Either supply a file to --eval_data_file "
            "or remove the --do_eval argument."
        )

    if (
        os.path.exists(my_args.output_dir)
        and os.listdir(my_args.output_dir)
        and my_args.do_train
        and not my_args.overwrite_output_dir
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                my_args.output_dir
            )
        )

    if my_args.server_ip and my_args.server_port:
        import ptvsd
        #print("Waiting for debugger attach")
        ptvsd.enable_attach(address=(my_args.server_ip, my_args.server_port), redirect_output=True)
        ptvsd.wait_for_attach()
    if my_args.local_rank == -1 or my_args.no_cuda:
        device = torch.device("cuda" if torch.cuda.is_available() and not my_args.no_cuda else "cpu")
        my_args.n_gpu = torch.cuda.device_count()
    else:  # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
        torch.cuda.set_device(my_args.local_rank)
        device = torch.device("cuda", my_args.local_rank)
        torch.distributed.init_process_group(backend="nccl")
        my_args.n_gpu = 1
    my_args.device = device
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if my_args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        my_args.local_rank,
        device,
        my_args.n_gpu,
        bool(my_args.local_rank != -1),
        my_args.fp16,
    )
    set_seed(my_args)
    if my_args.local_rank not in [-1, 0]:
        torch.distributed.barrier()  # Barrier to make sure only the first process in distributed training download model & vocab
    config_class, model_class, tokenizer_class = MODEL_CLASSES[my_args.model_type]
    #print ("\nLoadding model ", model_class)
    config = config_class.from_pretrained(
        my_args.config_name if my_args.config_name else my_args.model_name_or_path,
        cache_dir=my_args.cache_dir if my_args.cache_dir else None,
    )
    tokenizer = tokenizer_class.from_pretrained(
        my_args.tokenizer_name if my_args.tokenizer_name else my_args.model_name_or_path,
        do_lower_case=my_args.do_lower_case,
        cache_dir=my_args.cache_dir if my_args.cache_dir else None,
    )
    if my_args.block_size <= 0:
        my_args.block_size = (
            tokenizer.max_len_single_sentence
        )  # Our input block size will be the max possible for the model
    my_args.block_size = min(my_args.block_size, tokenizer.max_len_single_sentence)
    model = model_class.from_pretrained(
        my_args.model_name_or_path,
        from_tf=bool(".ckpt" in my_args.model_name_or_path),
        config=config,
        cache_dir=my_args.cache_dir if my_args.cache_dir else None,
    )
    model.to(my_args.device)
    if my_args.local_rank == 0:
        torch.distributed.barrier()  # End of barrier to make sure only the first process in distributed training download model & vocab
    logger.info("Training/evaluation parameters %s", my_args)
####################################################################################################
    # Training
    if my_args.do_train:
        if my_args.local_rank not in [-1, 0]:
            torch.distributed.barrier()
        train_dataset = load_and_cache_examples(my_args, tokenizer, logger, evaluate=False)
        if my_args.local_rank == 0:
            torch.distributed.barrier()
        global_step, tr_loss = train(my_args, train_dataset, model, tokenizer, logger)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)
        
    if my_args.do_train and (my_args.local_rank == -1 or torch.distributed.get_rank() == 0):
        if not os.path.exists(my_args.output_dir) and my_args.local_rank in [-1, 0]:
            os.makedirs(my_args.output_dir)
        logger.info("Saving model checkpoint to %s", my_args.output_dir)
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(my_args.output_dir)
        tokenizer.save_pretrained(my_args.output_dir)
        torch.save(my_args, os.path.join(my_args.output_dir, "training_args.bin"))

        model = model_class.from_pretrained(my_args.output_dir)
        tokenizer = tokenizer_class.from_pretrained(my_args.output_dir, do_lower_case=my_args.do_lower_case)
        model.to(my_args.device)

    # Evaluation
    results = {}
    if my_args.do_eval and my_args.local_rank in [-1, 0]:
        checkpoints = [my_args.output_dir]
        if my_args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(my_args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = model_class.from_pretrained(checkpoint)
            model.to(my_args.device)
            result = evaluate(my_args, model, tokenizer, logger, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)
            
    del model
    torch.cuda.empty_cache()
    
    return results

In [6]:
def fase_SC(from_pre, batch_si, d_lab, max_len):
    if from_pre:
        model_dir_ft = 'bert-base-uncased'
    else:
        model_dir_ft = '../../../../model_save/Dos-Fases-all_Harassment/'
    print('Loading BERT tokenizer...')
    
    tokenizer = BertTokenizer.from_pretrained(model_dir_ft, do_lower_case=True)
    print('Loading BERT Seq Class from ', model_dir_ft, '...' )
    model_loaded = BertForSequenceClassification.from_pretrained(model_dir_ft, num_labels=4) 
    model_loaded.cuda()

    print ("Modelo cargado correctamente desde ", model_dir_ft)
    df_read=pd.read_csv("../../CSV_Harassment/Train_data_compeition.csv")
    
    #etiq=[]
    #for et in df_read['Label'].values:
    #    etiq.append(d_lab[et])
    
    ys=df_read.values[:,2:]
    #y_train_mul=np.zeros((tr.shape[0],4))
    y_train=[]
    i=0
    for label in ys:
        if np.sum(label)==0:
            #y_train_mul[i][0]=1
            y_train.append(0)
        else: 
            #y_train_mul[i]=np.concatenate([[0],label[1:]])
            y_train.append(np.argmax(np.concatenate([[0],label[1:]])))
        i+=1
    
    sentences = df_read['tweet_content'].values
    n_labels = np.array(y_train)
    
    input_ids = []
    for sent in sentences:
        myc, myc_list= my_cleaner(sent)
        encoded_sent = tokenizer.encode(myc,add_special_tokens = True )    
        input_ids.append(encoded_sent)
        
    input_ids, attention_masks = make_padding_and_masks(max_len, tokenizer, input_ids)
    train_all, val_all = data_batches(input_ids, attention_masks, n_labels, test_size=0.1, batch_size=batch_si, mode='train')
    train_data, train_sampler, train_dataloader = train_all
    validation_data, validation_sampler, validation_dataloader = val_all

    optimizer = AdamW(model_loaded.parameters(), lr = 2e-5, eps = 1e-8)
    epochs = 1
    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

    seed_val = 42
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)
    loss_values = []
    model_loaded.zero_grad()    
    
    total_loss = 0
    model_loaded.train()        
    for step, batch in enumerate(train_dataloader):
        model_loaded.train()
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)                
        outputs = model_loaded(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_loaded.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        model_loaded.zero_grad()

    avg_train_loss = total_loss / len(train_dataloader)       
    loss_values.append(avg_train_loss)
    #print("  Average training loss: {0:.2f}".format(avg_train_loss))
    #print("Running Validation...")
    model_loaded.eval()
    eval_loss, eval_accuracy, eval_fscore = 0, 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():              
            outputs = model_loaded(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)        
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()     
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        tmp_eval_fscore = flat_fscore(logits, label_ids)
        eval_fscore += tmp_eval_fscore
        nb_eval_steps += 1
    #print("  F-score macro: {0:.2f}".format(eval_fscore/nb_eval_steps))
    #print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    #df_read=pd.read_csv("../CSV_Stance/dev_semeval_raw.csv")
    df_read=pd.read_csv("../../CSV_Harassment/testset-competition.csv")
    
    #etiq=[]
    #for et in df_read['Label'].values:
    #    etiq.append(d_lab[et])
    
    ys=df_read.values[:,2:]
    y_test=[]
    i=0
    for label in ys:
        if np.sum(label)==0:
            y_test.append(0)
        else: 
            y_test.append(np.argmax(np.concatenate([[0],label[1:]])))
        i+=1
  
    sentences = df_read['tweet_content'].values
    n_labels = np.array(y_test)
    
    input_ids = []
    for sent in sentences:
        myc, myc_list= my_cleaner(sent)
        encoded_sent = tokenizer.encode(myc,add_special_tokens = True )   
        input_ids.append(encoded_sent)
    input_ids, attention_masks = make_padding_and_masks(max_len, tokenizer, input_ids)
    pred_all = data_batches(input_ids, attention_masks, n_labels, test_size=0.1, batch_size=batch_si, mode='eval')
    prediction_data, prediction_sampler, prediction_dataloader = pred_all
    
    model_loaded.eval()
    predictions , true_labels = [], []
    for batch in prediction_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            outputs = model_loaded(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.append(logits)
        true_labels.append(label_ids)
    f_macros = []
    print('Calculating F-macro...')
    flat_predictions = [item for sublist in predictions for item in sublist]
    flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
    flat_true_labels = [item for sublist in true_labels for item in sublist]
    fma = f1_score(flat_true_labels, flat_predictions, average='macro')
    fno = f1_score(flat_true_labels, flat_predictions, average=None)
    acc = accuracy_score(flat_true_labels, flat_predictions)
    print('F macro: %.3f' % fma)
    print('F macro none average: ', fno)
    print('Accuracy: %.3f' % acc)
    
    state = {
            'device': device,
            'f_macro': fma,
            'F_none': fno,
            'Accuracy': acc,
            'epoch': epoch_macro,
            'batch_size': batch_si, 
            'max_len': max_len,
            'optimizer': optimizer ,
            'orden_epoch': 'primero'
        }
    save_model_to(out, model_loaded, tokenizer, state)
    
    del model_loaded
    torch.cuda.empty_cache()
    
    return fma, fno, acc

In [7]:
import gc
import resource

def mem():
    print('Memory usage         : % 2.2f MB' % round(
        resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1024.0/1024.0,1)
    )

In [8]:
Num_epochs= 5
out= '../../../../model_save/Dos-Fases-all_Harassment/'
from_pre = True

f= open(out+"/Summary_macro.txt","w")
f.write("This is a summary of the running \n")
f.close()

while epoch_macro <= Num_epochs:
    ####################################################
    ################## LANGUAGE MODEL ##################
    f=open(out+"/Summary_macro.txt", "a+")
    f.write("------------------------------------\n")
    f.write("Running Language Model epoch: "+str(epoch_macro)+'\n')
    resultado_lm = fase_LM(from_pre, my_args, logger)
    print ("Perplexity epoch: "+str(list(resultado_lm.values())[0].cpu().numpy())+'\n')
    f.write("Perplexity epoch: "+str(list(resultado_lm.values())[0].cpu().numpy())+'\n')
    f.close()
    ####################################################
    ################### Free memory ####################
    mem()
    print('GC collected objects : %d' % gc.collect())
    mem()
    
    gc.collect()    
    
    print('GC collected objects : %d' % gc.collect())
    mem()
    ####################################################
    ################### SEQUENCE CLASS #################
    f=open(out+"/Summary_macro.txt", "a+")
    f.write("------------------------------------\n")
    f.write("Running Sentence Classification epoch: "+str(epoch_macro)+'\n')
    
    from_pre = False
    
    #qscd
    d_lab=dict()
    d_lab['NonH']=0
    d_lab['IndirectH']=1
    d_lab['PhysicalH']=2
    d_lab['SexualH']=3
      
    max_len = 50
    batch = 16 
    fma, fno, acc = fase_SC(from_pre, batch, d_lab, max_len)
    f.write("F macro: "+str(fma)+'\n')
    f.write("F macro none avergage: "+str(fno)+'\n')
    f.write("Accuracy: "+str(acc)+'\n')
    f.write("------------------------------------\n")
    f.close()
    
    ####################################################
    ################### Free memory ####################
    mem()
    print('GC collected objects : %d' % gc.collect())
    mem()
    
    gc.collect()    
    
    print('GC collected objects : %d' % gc.collect())
    mem()
    
    epoch_macro += 1

03/05/2020 01:17:07 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
03/05/2020 01:17:08 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/mbugueno/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
03/05/2020 01:17:08 - INFO - transformers.configuration_utils -   Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "lab

Iteration:  64%|██████▍   | 99/154 [00:35<00:19,  2.78it/s]/home/casapanshop/anaconda2/envs/newpy3/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
03/05/2020 01:17:49 - INFO - transformers.configuration_utils -   Configuration saved in ../../../../model_save/Dos-Fases-all_Harassment/checkpoint-100/config.json
03/05/2020 01:17:51 - INFO - transformers.modeling_utils -   Model weights saved in ../../../../model_save/Dos-Fases-all_Harassment/checkpoint-100/pytorch_model.bin
03/05/2020 01:17:51 - INFO - __main__ -   Saving model checkpoint to ../../../../model_save/Dos-Fases-all_Harassment/checkpoint-100
03/05/2020 01:17:58 - INFO - __main__ -   Saving optimizer and scheduler states to ../../../../model_save/Dos-Fases-all_Harassment/checkpoint-100

Epoch: 100%|██████████| 1/1 [01:04<00:00, 64.01s/it]
03/05

03/05/2020 01:18:22 - INFO - transformers.tokenization_utils -   Didn't find file ../../../../model_save/Dos-Fases-all_Harassment/added_tokens.json. We won't load it.
03/05/2020 01:18:22 - INFO - transformers.tokenization_utils -   loading file ../../../../model_save/Dos-Fases-all_Harassment/vocab.txt
03/05/2020 01:18:22 - INFO - transformers.tokenization_utils -   loading file None
03/05/2020 01:18:22 - INFO - transformers.tokenization_utils -   loading file ../../../../model_save/Dos-Fases-all_Harassment/special_tokens_map.json
03/05/2020 01:18:22 - INFO - transformers.tokenization_utils -   loading file ../../../../model_save/Dos-Fases-all_Harassment/tokenizer_config.json
03/05/2020 01:18:22 - INFO - __main__ -   Evaluate the following checkpoints: ['../../../../model_save/Dos-Fases-all_Harassment/']
03/05/2020 01:18:22 - INFO - transformers.configuration_utils -   loading configuration file ../../../../model_save/Dos-Fases-all_Harassment/config.json
03/05/2020 01:18:22 - INFO - tra

Perplexity epoch: 51.809357

Memory usage         :  3.50 MB
GC collected objects : 427
Memory usage         :  3.50 MB
GC collected objects : 0
Memory usage         :  3.50 MB
Loading BERT tokenizer...
Loading BERT Seq Class from  ../../../../model_save/Dos-Fases-all_Harassment/ ...


03/05/2020 01:18:29 - INFO - transformers.modeling_utils -   Weights of BertForSequenceClassification not initialized from pretrained model: ['classifier.weight', 'classifier.bias']
03/05/2020 01:18:29 - INFO - transformers.modeling_utils -   Weights from pretrained model not used in BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']


...

Modelo cargado correctamente desde  ../../../../model_save/Dos-Fases-all_Harassment/

Padding/truncating all sentences to 50 values...

Padding token: "[PAD]", ID: 0
Completado.

Padding/truncating all sentences to 50 values...

Padding token: "[PAD]", ID: 0
Completado.


03/05/2020 01:33:00 - INFO - transformers.configuration_utils -   Configuration saved in ../../../../model_save/Dos-Fases-all_Harassment/config.json


Calculating F-macro...
F macro: 0.417
F macro none average:  [0.9160168  0.         0.         0.75277778]
Accuracy: 0.847
Saving model to ../../../../model_save/Dos-Fases-all_Harassment/


03/05/2020 01:33:02 - INFO - transformers.modeling_utils -   Model weights saved in ../../../../model_save/Dos-Fases-all_Harassment/pytorch_model.bin


Memory usage         :  3.80 MB
GC collected objects : 427
Memory usage         :  3.80 MB
GC collected objects : 0
Memory usage         :  3.80 MB
